# Download data from TCIA

This example Jupyter notebook demonstrates how to download data from The Cancer Imaging Archive (TCIA). Finally the data is visualised using the ImageVisualiser class within PlatiPy

### Import some functions to be able to download and visualise data

In [ ]:
try:
    import platipy
except:
    !pip install git+https://github.com/pyplati/platipy.git
    import platipy

import SimpleITK as sitk

from platipy.dicom.download.tcia import (
    get_collections,
    get_modalities_in_collection,
    get_patients_in_collection,
    fetch_data
)
from platipy.imaging.visualisation.tools import ImageVisualiser
from platipy.imaging.utils.tools import get_com

## Let's get a list of all collections available in TCIA

You can also browse the list of collections available here: https://www.cancerimagingarchive.net/collections/

In [ ]:
collections = get_collections()

## We'll use the 'Head-Neck Cetuximab' collection

In this cell we check what modalities are available with the collection

In [ ]:
collection = 'Head-Neck Cetuximab'
modalities = get_modalities_in_collection(collection)
print(f"Modalities available: {modalities}")


## Next let's see find the patients available in the collection

In [ ]:
patients = get_patients_in_collection(collection)

## Download the CT, RTSTRUCT and RTDOSE for the first patient

The fetch_data function will download the DICOM data from TCIA and will then convert it to NIFTI format for use within platipy.


In [ ]:
patient_id = patients[0]
data = fetch_data(
    collection,
    patient_ids=[patient_id],
    modalities=["CT", "RTSTRUCT", "RTDOSE"],
    nifti=True
)

## Finally, we load the NIFTI data and visualise it

In [ ]:
ct_path = [p for p in data[patient_id]["NIFTI"]["IMAGES"] if p.name.endswith("CT_1.nii.gz")][0]
ct = sitk.ReadImage(str(ct_path))

structures = {s.name.split(".")[0].split("_")[-1]: sitk.ReadImage(str(s)) for s in data[patient_id]["NIFTI"]["STRUCTURES"]}

dose = sitk.ReadImage(str(data[patient_id]["NIFTI"]["DOSES"][0]))
dose = sitk.Resample(dose, ct)


In [ ]:
img_vis = ImageVisualiser(ct, cut=get_com(structures["GTV"]), figure_size_in=10)
img_vis.add_scalar_overlay(dose, colormap="inferno", alpha=0.5)
img_vis.add_contour(structures)
fig = img_vis.show()